In [ ]:
def add_embeddings(self, query_idx, embeddings, pred_idx):
    '''
    query_idx : query indexes for current iteration
    embeddings : embeddings for query_idx
    pred_idx : predicted classes
    '''
    
    for i in range(len(pred_idx)):
        class_idx = pred_idx[i] #label
        img_idx = query_idx[i]
    
        self.support_database[class_idx]["img_idxs"].append(img_idx)
        self.support_database[class_idx]["embeddings"].append(embeddings[i])
        
    
def update_embeddings(self):
    
    for item in self.support_database.items():
        class_idx = item[0]   #key
        class_info = item[1]  #value
        images = []
        for img_ref in class_info["img_idxs"]:
            img = self.trans(self.vgg_dataset.__getitem__(img_ref)[0])
            images.append(img)

        aligned = torch.stack(images).to(device)
        embeddings = np.zeros((len(aligned), 512))

        # can keep as self.resnet maybe
        if resnet is None:
            resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

        for i in range(0, math.ceil(len(aligned)/32)):
            start = 32*i
            end = min(32*(i+1), len(aligned))
            embeddings[start:end] = resnet(aligned[start:end]).detach().cpu()

        embeddings = embeddings / np.linalg.norm(embeddings, axis=-1)[:, np.newaxis]
        self.support_database[class_idx]["embeddings"] = embeddings.detach().cpu()

        prototype_embedding = np.mean(embeddings, dim=0)
        self.support_database[class_idx]['prototype'] = prototype_embedding
    
    